# Step 1
Carry over the S3 bucket stored from the first notebook and import dependencies

In [ ]:
import boto3
import io
import numpy as np
import json
import random
import pandas as pd
import matplotlib.pyplot as plt

random.seed()

%store -r
print ('tf_endpoint_name=',tf_endpoint_name)
print ('data_bucket_name=',data_bucket_name)

# Step 2
Define Labels for activities

In [ ]:
ACT_LABELS = ["downstairs","upstairs", "walking", "jogging", "standing", "sitting"]

# Step 3
Supporting Function to load S3 file into a numpy object

In [ ]:
def load_data(bucket_name,key):
    s3_client = boto3.client('s3')
    obj = s3_client.get_object(Bucket=bucket_name, Key=key)
    
    with io.BytesIO(obj["Body"].read()) as f:        
        f.seek(0)
        data = np.load(f)
    return data

# Step 4
Load S3 files into textX and testY numpy arrays.  textX is the sensor data; testY is the one-hot encoded label for ground truth.  In this notebook, this allows us to compare the predicted class to actual class.

In [ ]:
testX = load_data(data_bucket_name,'test/test_data.npy')
testy = load_data(data_bucket_name,'test/test_labels.npy')

# Step 6
Reshape the data into a multi-dimension array of steps and length of step.  As delivered, the original length is 128 steps and these were trained in 4x32.  This code is editable, if you understand the implictations; it works as delivered.

In [ ]:
n_features, n_outputs = testX.shape[1], testy.shape[1]
n_steps, n_length = 4, 32
input_data = testX.reshape((testX.shape[0], n_steps, n_length, n_features))

# Step 7
Select a random record out of the set, convert the data into JSON and submit to the endpoint for a real-time prediction.  The output will print each class, the probability of being a member of each class and the true class from the testy object.

In [ ]:
r = random.randint(0,input_data.shape[0])
print('random record=',r,)

inference_input=input_data[r:r+1]
print('inference_input=',inference_input.shape)

ep_name = tf_endpoint_name

client = boto3.client('sagemaker-runtime')

payload=inference_input.tolist()

response = client.invoke_endpoint(EndpointName=ep_name,
                                   ContentType='application/json',
                                   Body=json.dumps(payload)
                                 )

result = json.loads(response['Body'].read().decode())

predictions=result['predictions'][0]

for i in range(0,len(predictions)):

    print('class=',ACT_LABELS[i], 'proba=',round(float(predictions[i]),4), 'truth=',testy[r][i])


# Step 8
Optionally, look at the random data row in a tabular fashion

In [ ]:
x = testX[r]
df = pd.DataFrame(x[0])

for i in range(1,n_features):
    df.insert(i, i, x[i], True)
    
df.columns=(['attitude.roll','attitude.pitch','attitude.yaw','gravity.x','gravity.y','gravity.z','rotationRate.x','rotationRate.y','rotationRate.z','userAcceleration.x','userAcceleration.y','userAcceleration.z'])

df

# Step 9
If you completed step 8, you may visualize the sensor data as time-series movement too across steps.  As delivered, step length is 128, or about 2.5 seconds of activity.

In [ ]:
df.plot(subplots=True,figsize = (20,30))
plt.show()